### Initialize Model 

In [1]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import sqlite3

# Recall your saving path
save_directory = "./local_gpt2_model"


# Load the model
model = GPT2LMHeadModel.from_pretrained(save_directory)

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(save_directory)

def initialize_db(db_name="context_DPM_data.db"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    # Create a table to store contexts
    cursor.execute('''CREATE TABLE IF NOT EXISTS contexts
                     (id INTEGER PRIMARY KEY, context TEXT)''')
    conn.commit()
    return conn, cursor

def insert_context(conn, cursor, context):
    cursor.execute("INSERT INTO contexts (context) VALUES (?)", (context,))
    conn.commit()

def generate_answer_gpt2(question, context, max_length=500):
    input_text = f"{context}. {question}"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    
    # Generate a response from the model
    outputs = model.generate(input_ids, max_length=max_length, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id, no_repeat_ngram_size=2, top_k=50, top_p=0.95)
    
    # Decode the output
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # The answer is whatever was added by the model, after the input text
    answer = generated_text[len(input_text):].strip()
    
    return answer

def retrieve_relevant_contexts(question, cursor, limit=10):
    # Extract keywords from the question (this is a simple approach, can be improved)
    keywords = question.split()
    
    # Query the database to find contexts that contain the keywords
    contexts = []
    for keyword in keywords:
        cursor.execute("SELECT context FROM contexts WHERE context LIKE ?", ('%' + keyword + '%',))
        contexts.extend([row[0] for row in cursor.fetchall()])
        
    # Return unique contexts and limit the number of results
    return list(set(contexts))[:limit]

def answer_question_gpt2_v2(question, cursor):
    # Fetch the most relevant contexts for the question
    contexts = retrieve_relevant_contexts(question, cursor)
    
    # If no context found, provide a default answer
    if not contexts:
        return "I don't have enough information to answer that question."
    
    # Loop through each context and generate an answer using GPT-2
    for context in contexts:
        answer = generate_answer_gpt2(question, context)
        if answer:
            return answer
    return "I couldn't generate a satisfactory answer based on the provided contexts."

# Connect to DB
conn, cursor = initialize_db()

/usr/local/lib/python3.7/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [10]:
def get_contexts_for_question(conn, cursor, question):
    # Extract keywords from the question
    keywords = question.split()
    
    # Construct the query to search for contexts containing keywords
    query_clauses = ["context LIKE ?" for _ in keywords]
    query = "SELECT context FROM contexts WHERE " + " OR ".join(query_clauses)
    params = ['%' + keyword + '%' for keyword in keywords]
    
    cursor.execute(query, params)
    results = cursor.fetchall()
    
    # Extract contexts from the results
    contexts = [result[0] for result in results]
    
    return contexts

In [17]:
# Ask a question and retrieve relevant contexts
question = "Liquidated Damages"
contexts = get_contexts_for_question(conn, cursor, question)

print("Found " + str(len(contexts) + " Contexts"))
for context in contexts:
    print(context)
    print("--------------------------------------------")

Found contexts:
Imposition of LD While Granting Extensions: While granting extensions of delivery period on an application of the contractor, the letter and spirit of the application should be kept in view in fixing a time for delivery and it must be decided while granting extension whether it would be with or without imposition of Liquidated Damages.
Where deliveries are accepted beyond the scheduled Delivery Date subject to levy of liquidated damages as provided in the Contract, the liquidated damages (if a percentage of the price) will be applicable on the price as varied by the operation of the Price Variation Clause.
Liquidated Damages (LD)
Liquidated damages: Compensation of loss on account of late delivery where loss is pre-estimated and mutually agreed to is termed as the Liquidated Damage (LD). Law allows recovery of pre-estimated loss, provided such a term is included in the contract. For imposition of LD, there is no need to establish actual loss due to late supply. The lega

In [19]:
from transformers import top_k_top_p_filtering
import torch
import torch.nn.functional as F

def answer_question_using_context(model, tokenizer, context, question, max_length=150):
    input_text = context + " " + question
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    
    generated_ids = input_ids.clone()
    token_probs_list = []

    with torch.no_grad():
        for _ in range(max_length - len(input_ids[0])):
            logits = model(generated_ids).logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            next_token_logits = top_k_top_p_filtering(logits, top_k=50, top_p=0.95)
            next_token = torch.multinomial(F.softmax(next_token_logits, dim=-1), num_samples=1)
            generated_ids = torch.cat((generated_ids, next_token), dim=-1)
            
            # Get the token's probability and store
            next_token_prob = probs[0, next_token.item()].item()
            token_probs_list.append(next_token_prob)
    
    # Compute average confidence
    confidence_score = sum(token_probs_list) / len(token_probs_list) if token_probs_list else 0
    
    answer = tokenizer.decode(generated_ids[0][len(input_ids[0]):], skip_special_tokens=True)

    return answer, confidence_score

In [20]:
# Ask a question and retrieve relevant contexts
question = "What are Liquidated Damages?"
contexts = get_contexts_for_question(conn, cursor, question)

# Answer the question using each context
for context in contexts:
    answer, confidence = answer_question_using_context(model, tokenizer, context, question)
    print(f"Context: {context}")
    print(f"Answer: {answer}")
    print(f"Confidence Score: {confidence:.4f}\n")
    print("-------------------------------------")


Context: The principles and procedures contained in this Manual are to be followed by all wings of the Ministry of Defence and the Defence Services, as well as all organizations and units/establishments thereunder, for procurement of goods and services, expenditure on account of which is met from the revenue heads of the Defence Services Estimates (DSE) and any other type of purchases to which the provisions are made specifically applicable.
Answer:  The General Defence Regulations Act 1977 ("The Defence Services Regulations") states that a liquidated damages claim must be made by a person for any injury to a person resulting from a breach of the provisions of this Manual. This means that a person seeking to obtain a liquidated damages award must present proof of the injury to be
Confidence Score: 0.3165

-------------------------------------
Context: The term procurement means acquiring all types of goods (both scaled and non-scaled), such as equipment, stores, spares, technical liter

KeyboardInterrupt: 

In [16]:

# Ask a question and retrieve relevant contexts
question = "What are Liquidated Damages?"
contexts = get_contexts_for_question(conn, cursor, question)

answers_with_confidence = []

# Answer the question using each context and store the answers and confidence scores
for context in contexts:
    answer, confidence = answer_question_using_context(model, tokenizer, context, question)
    answers_with_confidence.append((answer, confidence, context))

# Sort by confidence and get the answer with the highest score
sorted_answers = sorted(answers_with_confidence, key=lambda x: x[1], reverse=True)
top_answer, top_confidence, top_context = sorted_answers[0]

print(f"Context with highest confidence: {top_context}")
print(f"Answer with highest confidence: {top_answer}")
print(f"Confidence Score: {top_confidence:.4f}\n")
print("-------------------------------------")

NameError: name 'F' is not defined

In [18]:
# Ask a question and retrieve relevant contexts
question = "What are Liquidated Damages?"
contexts = get_contexts_for_question(conn, cursor, question)

# Answer the question using each context
for context in contexts:
    answer, confidence = answer_question_using_context(model, tokenizer, context, question)
    print(f"Context: {context}")
    print(f"Answer: {answer}")
    print(f"Confidence Score: {confidence:.4f}\n")
    print("-------------------------------------")


NameError: name 'F' is not defined

In [22]:
# Ask a question and retrieve relevant contexts
question = "What is Force Majeure?"
contexts = get_contexts_for_question(conn, cursor, question)

# Answer the question using each context
for context in contexts:
    answer, confidence = answer_question_using_context(model, tokenizer, context, question)
    print(f"Context: {context}")
    print(f"Answer: {answer}")
    print(f"Confidence Score: {confidence:.4f}\n")
    print("-------------------------------------")


Context: FORMAT FOR ISSUING SANCTIONS 328
Answer:  328.1 Force majeure is defined in Chapter 3-4 of Part VIII of this Schedule as when, in the opinion of any person or authority in his place of duty, any emergency arises or any action, failure to perform or any failure in a particular duty is reasonably likely to arise. A person or authority shall, in exercising his or her power, exercise due care to ascertain whether any particular situation and or duty presents any such emergency. In the absence of adequate warning or justification, the officer shall not attempt to rescue or render aid unless there are sufficient alternative means of escape available to him, or, failing such alternative, unless that person or authority is
Confidence Score: 0.2430

-------------------------------------
Context: This Manual may be called the Defence Procurement Manual, 2009 (DPM 2009, for short).
Answer:  To answer this question, we first need to understand how force majeure is defined under Canadian l

### Old Questions

In [8]:
question = "What are the different types of officers in the Indian Navy?"
answer = answer_question_gpt2_v2(question, cursor)
print(answer)

(a) The Chief of the Navy.
 (b) Chief, Chief Engineer, Commander, Captain, Lieutenant, Major, and Captain. 
What are some of these officers?  What is the role of each of them? What does the Chief do? How does he operate? Is he the head of any department? Does he have any authority to make decisions? Do he make any decisions himself? Are there any rules or regulations? Who is responsible for the performance of his duties? Where does his authority come from? Why does it exist? And what is his role in relation to the other officers of this class? (c) Captain and Lieutenant. (d) Lieutenant and Major.  What is their role? 
How does a Lieutenant Commander operate in this role, or does that role come under the authority of a Captain?  
Is there a hierarchy of rank within the Naval Service? If so, what does this hierarchy consist of? Can a Commander be a Major? A Lieutenant? a Colonel? or a Vice Admiral? The answer to these questions will determine the type of officer that the Indians will hav

In [9]:
question = "What is the role of the audit officer according to chapter 4?"
answer = answer_question_gpt2_v2(question, cursor)
print(answer)

1. The audit officers are responsible for the following:
a) The preparation of a statement of financial statements and accounts of all the entities and persons who are
i) the subject of audit.
ii) who have been audited by the auditing agency. They are also responsible
iii) for ensuring that the financial statement and account of each entity and person is
iv) accurate and complete. In addition, they are required to
v) prepare a report on the results of their audit and to make recommendations to the
vi) auditors. 2. Each audit agency has a separate audit division. This division is responsible to prepare
3. A statement is prepared by each audit department. It is a summary of information
4. These statements are submitted to each auditor. 5. When the auditor receives the statement, he or she must
6. make a decision on whether to accept or reject the report. 7. If the decision is to reject, the Auditor must send a copy of his or her decision to all
8. audit officers. 9. After the final decis

In [10]:
question = "What are the Prefeaces of Defence Audit Maunal?"
answer = answer_question_gpt2_v2(question, cursor)
print(answer)

(a) The Preference of the Defence Auditor General in the preparation of a report on the performance of defence audit Mauna Kea.
 (b) A statement of all the information that the Auditor-General has received from the Government of New Zealand in relation to the audit of Māori Defence. (c) An explanation of how the auditor-general has been able to obtain the relevant information.  What is the purpose of this report? 
The purpose is to provide the public with a detailed account of what has happened in Mōori defence and the impact of that on Mīori people. The report will also provide an opportunity for the Minister to respond to questions raised by the media. This report is not intended to be a substitute for a full audit report. It is intended as a supplement to that report and will not be used as the basis for any subsequent audit.


In [11]:
question = "The Comptroller and Auditor General of India today derives his authority from where?"
answer = answer_question_gpt2_v2(question, cursor)
print(answer)

(a) Section 4 of the Public Accounts (Management) Act, 1956, as amended by the Finance Act of 2012, and
 (b) section 4(1) of that Act.
, the Comsec. of Finance has been empowered to make such payments as may be necessary to meet the obligations of, or to pay the interest on, such debt or deposits. In the case of a debt of Rs. 1,000 crore or more, it may, in the exercise of its powers under this section, make payments of up to Rs 1 lakh crore. It may also make payment of interest at a rate of 5 per cent per annum. Such payments may not be made on account of any other debt. If the debt is of less than Rs 500 crore, payment may only be on the basis of an amount equal to the amount of such interest. Payment may take place on any day of payment. Payments may include interest only on such amount as is due. Any payment made under the provisions of this Section shall be deemed to be a payment under section 5 of Public Finance (Procedures) Rules, 1952. (a), (c) and (d) are not applicable to any

In [13]:
question = "What are the fundamental Principles of Public Buying according to CHAPTER 2 PROCUREMENT – OBJECTIVE AND POLICY?"
answer = answer_question_gpt2_v2(question, cursor)
print(answer)

(a) The objective of the Public Purchasing is to obtain the necessary funds for the maintenance of public services and the provision of goods and services to the public.
 (b) Public purchasing is carried out in accordance with the following principles:
1. The public purchasing of services is done in the interests of all the citizens of a State. 2. Public buying is conducted in a manner that is fair and equitable. 3. All the goods, services, facilities and facilities of State institutions are purchased in good faith. 4. Purchases are made in such a way that the cost of purchasing the services of state institutions is borne by the State and not by any individual. 5. In the case of private purchases, the costs of procurement are borne in proportion to their value. 6. When the price of any service is fixed, it is not possible to determine the value of that service. 7. It is the duty of every State to ensure that all its public institutions, public agencies and public undertakings are able 

In [14]:
question = "What is importance of fund availability when doing public procurement according to DPM 2009?"
answer = answer_question_gpt2_v2(question, cursor)
print(answer)

1. The fund is available for public procurements when the total amount of the fund available to the public is at least Rs. 1 crore.
2. When the amount available is less than Rs 1 crores, the government may not be able to procure the goods or services. In such case, it may be necessary to seek the assistance of private sector. 3. If the funds available are less then Rs 2 crore, then the Government may seek private assistance. 4. It is important to note that the Public Procurement Act, 2009 (PPA) provides for the provision of funds for Public procurement. 5. Public procurement is a public purpose and the purpose of this Act is to provide for procurement of goods and services for which the cost of procurement is borne by the State. 6. This Act provides that funds may only be provided for a period of one year from the date of commencement of public procurements. 7. A public body may, in its discretion, provide funds to a private body for any purpose. 8. Funds provided under this section ma

In [15]:
question = "WHat is the procedure for Foreign Procurement iaw Defence Procurement Manual 2009?"
answer = answer_question_gpt2_v2(question, cursor)
print(answer)

1. The Foreign procurement manual 2009 is available from the Ministry of Defence website at: http://www.mofa.gov.uk/pubs/fpg/pdf/Fpg2009.pdf
2. Where a foreign procurement contract is awarded, the contract must be signed by the contracting party and the foreign procurer.
3. A foreign procuree must submit a copy of the Foreign procuree's contract to the Contracting Officer. If the FPO is not available, a Contractor's Representative may be appointed. This Contract Officer will then sign the document.  The Contractors' Representative will also sign a statement of receipt, which must include the name of each foreign supplier and their address. 
4. When the procurement is completed, it is sent to a Foreign Contract Office for approval. Once approved, this contract will be sent back to FCO for further approval and approval of further contracts. FOCO will send the final contract back for final approval to all foreign suppliers. In the case of a contract for a single product, FODO may also sen

In [16]:
question = "What are various modes of public Procurement in Defence?"
answer = answer_question_gpt2_v2(question, cursor)
print(answer)

(a) The Defence Procuresment Act, 1952 (as amended) provides that the Defence Minister may procure goods and services for the
    (i) Defence Forces,
   (ii) the Royal Australian Air Force, and
    (iii) other Defence Force units and units of the Australian Defence
            Force.
The Defence Act provides for a number of different ways in which the Minister can procure
     
1. goods for use in the defence of Australia. The Act also provides the following:
" 1.1 The Minister shall procure for Australia the goods, services and facilities necessary for
                                                the performance of its duties under this Act. 1.2 The goods or services may be
for the purpose of providing for or supporting the operation of a Defence force unit or unit of an Australian defence force. 2.2.3 The defence goods may include goods that are not for sale or for resale. 3.4 The minister may not procure any goods unless the minister is satisfied that they are necessary to meet t

In [20]:
question = "What is single tender enquiry with respect to Public procurement as per Defence Procurement Manual in India?"
answer = answer_question_gpt2_v2(question, cursor)
print(answer)

1. The Defence procurement policy is to procure only the most necessary items for the country's defence needs.
2. In the case of procurement of defence items, the Government of India has to make a decision on the cost of the item and the amount of cost. This decision is made by the Defence Minister. It is the responsibility of each department to submit a report to the Minister on its decision. If the decision of a department is not in conformity with the policy of Defence, then the department has the right to appeal to a higher authority. However, if the Department of Defense does not submit its report within a reasonable period of time, it is considered as a failure of its duty. 
3. Under the single- tender policy, a single department can submit an enquiries report on a particular item. A single enquisition report is a written report which is submitted to all departments. Each department may submit one enquiring report. An enquirer is an individual who is appointed by a Defence Minist

In [24]:
question = "What are the different types of submarines in the Indian Navy?"
answer = answer_question_gpt2_v2(question, cursor)
print(answer)

(a) Submarines. The Indian Naval Submarine Force (INS) consists of the following types:
- Sub-marine (SSM) - Submersible (SMS)
Sub-marine ( SSM ) is a submersable submarine that is designed to operate in shallow water and is capable of operating in a variety of weather conditions. It is the most common type of submarine in Indian service.
The submarine is equipped with a crew of four and can carry up to eight people. Submerged in water, the submarine can operate for up a period of up, 30 minutes. In the case of a submarine sinking, it is possible to rescue the crew by using a rescue helicopter. A rescue chopper can be used to reach the submerged submarine. This type is also used for rescue operations in deep water. (b) Surface Combatant (SC) and Sub Surface (ST) submarines. These submarines are designed for surface combatant and sub surface operations. They are equipped to carry out surface warfare and are capable to engage in surface-to-surface combat. SC and ST submarines can also op

In [25]:
question = "Generate email for the Indian Navy saying i am sick?"
answer = answer_question_gpt2_v2(question, cursor)
print(answer)

1.  Send a copy of the letter to the following address:
2. Send a letter of explanation to: 
3. Write to  The Indian Government of India
4. Make a statement of your intention to pay the amount due to us.
5. Submit the statement to :
6. The Secretary of State for External Affairs
7. Indian Embassy in Washington, DC
8. Department of Foreign Affairs,
9. Embassy of Pakistan
10. Ministry of External affairs
11. Foreign Ministry
12. Government Of India, Delhi
13. Minister of Home Affairs of Indian Republic of Kashmir
14. State Department
15. Secretary General of United Nations
16. United States Department Of State
17. Office of International Cooperation
18. Bureau of Consular Affairs and Consulate General
19. Consulates General in India and Pakistan.


In [30]:
question = "Who was Romeo?"
answer = answer_question_gpt2_v2(question, cursor)
print(answer)

Romeo was the son of a wealthy merchant who had been a member of Parliament for many years. He was born in 1848 in a village in
the village of Bhatkal, in Bihar, India. His father was a merchant and his mother was an educated woman. Romeo was educated at the University of
India and at a private school in Delhi.
He was appointed as a clerk in Parliament in 1860 and was elected as an MP in 1861. In 1866 he was
elected as the MP for the constituency of Kolkata. On the same day he became the first MP to be elected from a
district in India to represent the district of Delhi, and he served as MP from 1867 to 1871. From 1872 to
1875 he represented the seat of Thane in Bombay. After the death of his father in 1880, Romeo married
his second wife, a woman named Anjali. They had two children, one of whom was named Romeo. The second Romeo died in 1890. It was during this period that Romeo became a Member of parliament. During his
life he had many engagements and engagements were arranged for him b

In [31]:
question = "Who was Juliet?"
answer = answer_question_gpt2_v2(question, cursor)
print(answer)

Juliet was the daughter of Sir John Juliet, who was
the first Governor of New South Wales. She was born in 1835, and was educated at
her father's school in Sydney, Australia. Her father was a lawyer, but she was also a
womanizer, a lover of dancing, an amateur dancer, dancer and a dancer. Juliet married
John Henry, Lord of Sydney in 1840, after which she moved to London. In 1846 she married Sir
William Henry. They had two children, one of whom was Sir William Henry's son, Sir Henry
Henry. Sir Richard Henry was married to Lady Jane Grey, daughter to Sir Thomas Grey.
In 1848, Juliet moved with her husband to England, where she became a solicitor. After a few years, she returned to Australia,
where she lived until her death in 1860. The following year, her son Sir Edward
Juliet was appointed Governor-General of Australia by the King. He was succeeded by Sir George
Edward, Governor for the Northern Territory. His son was Governor
of New Zealand. During the reign of Queen Victoria, Victoria

In [32]:
question = "What is Wikipedia?"
answer = answer_question_gpt2_v2(question, cursor)
print(answer)

It is a collection of articles, which are organized into a series of pages. The articles are divided into categories, and each category is divided in a different way.

The first category, "General Information", is the most basic. It contains the basic information about the subject of the article. This is what Wikipedia is about. Wikipedia's purpose is to provide a comprehensive and up-to-date overview of all the information available on the Internet. In this category are the articles on Wikipedia, the Wikipedia encyclopedia, Wikipedia articles and Wikipedia pages, as well as the related Wikipedia-related articles. These articles provide information on topics such as Wikipedia and the internet, or on subjects such a Wikipedia article, a wiki article or a wikipedia page. For example, an article on wikis, such an Wikipedia page, is an encyclopedia article and is therefore a part of Wikipedia. An article about Wikipedia can be a page on a website, on an online encyclopedia or even on anoth

In [33]:
question = "Who is Divyesh?"
answer = answer_question_gpt2_v2(question, cursor)
print(answer)

He is the one who is responsible for the

disappearance of the entire population of India. He has been
, and is still, the chief of this country.
The fact that he is a man of great power and influence is not
...
(The writer is speaking of his own personal experience.)
"The people of Delhi are not the only ones who are suffering from
the absence of their leaders. The people in the other states of
India are also suffering. They are the ones whose leaders are
not there. It is because of these people that the country is
in a state of crisis. This crisis is caused by the absence
of leaders in Delhi. If the leaders of a country are absent, it is only
because of some other cause. But if the people are deprived of leaders,
it is due to some cause which is beyond the control of any
person. And this cause is always the same. That cause must be
unjust. In the case of an unjust cause, there is no remedy. There is nothing
that can be done about it. So, if there are no leaders to take care of it, th

In [34]:
question = "Who is SRK?"
answer = answer_question_gpt2_v2(question, cursor)
print(answer)

Who are the people who are responsible for this?

The answer is that the SRM is the only one who is responsible.
,
.


In [35]:
question = "Who is Salman Khan?"
answer = answer_question_gpt2_v2(question, cursor)
print(answer)

He is a man of great talent, who has been

in the public eye for a long time. He has a great reputation for his
, and his films. His films are not only popular, but they are also
.
The fact that Salman is not a household name is also a fact. The fact
that he is an actor is another fact, which is why he has to be
treated with respect. It is the same with his movies. They are
not just popular. There are many people who are watching them. But
they are only watching Salman. And he must be treated with the respect
which he deserves. If he does not, he will not be able to do his job. I
am not saying that he should be given a lot of money, or that his name should
be attached to a movie. What I am saying is that the fact of his being an
actor is irrelevant. Salman has done his work. You cannot say that
he is doing his own work, because he did not do it. That is what I mean by
"not doing it".
I am not going to say anything about Salman's family. All I can say is this. We
have to respect him. Th

In [36]:
question = "What is the percentage of ld?"
answer = answer_question_gpt2_v2(question, cursor)
print(answer)

(a) In the case of a debt of $1,000, the amount of the debt is $100.
 (b) in the other case, it is 50%.
 (c) The amount in (a), (b), and (d) is to be added to the total amount paid by the taxpayer in respect of debt. (2) Where the Minister has received a statement from the Receiver General under section 7 of this Act, he shall, on receipt of it, give notice of his intention to make a payment under this section. The Minister shall give the notice to each person who is entitled to receive it. If the person is not entitled, or is unable to, to pay the money, then the payment shall be made to that person. Where a person has paid a sum of money to a Receiver under subsection (1), the amounts paid shall not be deemed to have been paid to any other person, and the same shall apply to all payments made under that subsection. R.S., 1985, c. 1 (5th Supp.), s. 7; 1996, f. 3, s, 1; 1999, g. 2, Sched. A, ss. 5, 6; 2001, C. 15, Sch. II, p. 4; 2002, P.A. 83, § 1, eff. July 1.


In [ ]:
question = "What is LD as per DPM 2009?"
answer = answer_question_gpt2_v2(question, cursor)
print(answer)